# Artificial Intelligence
## Assignment 1 – Search

### Personal details

* **Name:** YOUR NAME HERE
* **Student ID:** YOUR STUDENT ID HERE

In this assignment, you will implement a number of search algorithms to solve a simple navigation problem. We will explore both uninformed and informed search strategies, and you will have the opportunity to compare their performance.


### 0.0 – Prerequisites

There are some concepts you should be familiar with before going ahead with the assignments for this course. There are no tasks in this first section – all the code is provided for you to read and understand. If you're already familiar with Python classes as well as the stack and queue data structures, you can skim through this section. If not, please read the following explanations carefully.

Python is an **object-oriented** programming language and as such allows you to define your own **classes**, which can be utilized for implementing abstract data types. A class is defined using the `class` keyword, followed by the name of the class (typically in CapWords) and a colon. A class can contain **attributes** (member variables) and **methods** (member functions) that define its behavior. In this course, we define all member variables in the `__init__` method and initialize them as `None` if needed, as Python does not support uninitialized variables.

A **stack** is a data structure that follows the Last In First Out (LIFO) principle, meaning that the last element added to the stack is the first one to be removed. It can be visualized as a vertical stack of books, where you can only add or remove books from the top of the stack. A stack is defined by three operations: 1) `read`: you may read the *last* element of the stack, 2) `push`: you may insert data at the *end* of the stack and 3) `pop`: you may delete (and read) data from the *end* of the stack. No other operations for manipulating data are permitted.

A stack can be implemented in the following way using a class and a list:

In [1]:
class Stack:
    """
    A simple stack implementation (LIFO: Last In, First Out).
    """
    def __init__(self):
        self._data = []

    def push(self, element):
        """Add an element to the top of the stack."""
        self._data.append(element)

    def pop(self):
        """Remove and return the top element, or None if empty."""
        if len(self._data) > 0:
            return self._data.pop() # note that pop() is also a built-in method for lists and has a default argument of -1,
                                    # which removes and returns the last item
        else:
            return None

    def read(self):
        """Return the top element without removing it, or None if empty."""
        if len(self._data) > 0:
            return self._data[-1]
        else:
            return None

    def is_empty(self):
        """Return True if the stack is empty, False otherwise."""
        return len(self._data) == 0

Some notes:

* The `__init__` method is a **constructor** that is automatically called when an object of the class is created. It initializes the attributes of the class. Here, it initializes an empty list to represent an empty stack.
* The underscore before the attribute name `_data` is a convention used in Python to indicate that it is a protected attribute, meaning it should not be accessed directly from outside the class. Instead, you should use the provided methods to interact with the stack. This corresponds with using the `protected` keyword in other programming languages like Java or C++. Double underscores (e.g., `__data`) would indicate a private attribute, but this is not so common in Python.
* Each method in the class has a `self` parameter, which refers to the instance of the class. This allows you to access the attributes and methods of the class. Attributes and methods of the class *inside* the class are accessed using the `self` keyword, while attributes and methods of the class *outside* the class are accessed using the instance name.
* For simplicity, the `pop` method is designed to also return the last element of the stack, making `read` only necessary for debugging purposes.

Here is how you would use the `Stack` class:

In [2]:
my_stack = Stack() # create an instance of the Stack class we just defined
my_stack.push(1) # stack is now [1]
my_stack.push(2) # stack is now [1, 2]
print(my_stack.read()) # should print 2
x = my_stack.pop() # stack is now [1], popped element (2) is saved in x
print(my_stack.read()) # should print 1
print(my_stack.pop()) # pops and prints 1 without saving it, stack is now empty
print(my_stack.read()) # should print None
print(my_stack.is_empty()) # should print True
print(x) # should print 2, which we saved earlier

2
1
1
None
True
2


A **queue** is a data structure that follows the First In First Out (FIFO) principle, meaning that the first element added to the queue is the first one to be removed. It can be visualized as a line of people waiting at the grocery store, where the first person in line is the first one to be served. A queue is defined by three operations: 1) `read`: you may read the *first* element of the queue, 2) `enqueue`: you may insert data at the *end* of the queue and 3) `dequeue`: you may delete (and read) data from the *front* of the queue. No other operations for manipulating are permitted.

A queue can be implemented in the following way using a class and a list:

In [3]:
class Queue:
    """
    A simple queue implementation (FIFO: First In, First Out).
    """
    def __init__(self):
        self._data = []
    
    def push(self, element): # enqueue, we use push/pop to keep the naming consistent with Stack
        """Add an element to the end of the queue."""
        self._data.append(element)

    def pop(self): # dequeue
        """Remove and return the front element, or None if empty."""
        if len(self._data) > 0:
            return self._data.pop(0)
        else:
            return None
    
    def read(self):
        """Return the front element without removing it, or None if empty."""
        if len(self._data) > 0:
            return self._data[0]
        else:
            return None
        
    def is_empty(self): # helper method for later
        """Return True if the queue is empty, False otherwise."""
        return len(self._data) == 0

Here is how you would use the `Queue` class:

In [4]:
my_queue = Queue() # create an instance of the Queue class we just defined
my_queue.push("a") # queue is now ["a"]
print(my_queue.is_empty()) # should print False
my_queue.push("b") # queue is now ["a", "b"]
print(my_queue.read()) # should print "a"
x = my_queue.pop() # queue is now ["b"], "a" is saved in x
print(my_queue.read()) # should print "b"
print(my_queue.pop()) # also prints "b", stack is now empty
print(my_queue.read()) # should print None
print(x) # should print "a", which we saved earlier

False
a
b
b
None
a


You will need to use the `Stack` and `Queue` classes in the tasks below. Make sure you understand how they work before proceeding.

### 1.1 – Uninformed Search

![gridgame.png](gridgame.png)

<small>Image generated with Le Chat</small>

Imagine a third-person video game where you control a character that can move freely in a 3D world. The same is true for other characters in the game, which are controlled by the computer. Let's say one of these computer-controlled characters is trying to reach the player, but the way is not clear as there are obstacles in the way. How would movement and path-finding in the game be implemented?

An important point is that a virtual 3D world does not require omni-directional movement or a continuous space. Assuming a sufficiently flat environment, the space of allowed locations can be represented as a discrete 2D **grid**. This grid can in turn be interpreted as a **graph** where each cell in the grid is a **node** and the edges are the connections between adjacent nodes.

In order to apply a search algorithm to this problem, we need to define the following:

* **Search space**: the set of all possible states that can be reached from the initial state. In our case, this is the set of all nodes in the grid.
* **Initial state**: the state from which the search starts.
* **Goal state**: the state that we want to reach.
* **Actions**: the set of all possible actions that can be taken from a given state. In our case, this is the set of all possible moves that can be made from a given square (up, down, left, right).
* **Action cost**: each action has a cost associated with it, which is the cost of moving from one node to another. In this problem we are working with an unweighted grid, so the cost of each action is the same.
* **Transition model**: a function that takes a state and an action and returns the resulting state. In our case, since there is no uncertainty associated with the actions, the transition model is simply the action itself (i.e., moving to an adjacent node).

The first thing we need to do is to define appropriate data structures for the problem. We've already defined the `Stack` and `Queue` classes above, but we still need a way to represent the grid and the nodes in it. We can define the grid as a list of lists, where each element is a `Node` object. You may take a look at the `ex1_utils.py` file to see how the grid is constructed and populated with nodes, but this is not strictly necessary.

In [5]:
from ex1_utils import Grid, Node

In [6]:
grid = Grid()
grid.generate_nodes()
grid.visualize()

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . ■ ■ ■ ■ . . . . . . . . . . . . . . . . . . . . . . . . . . . . ■ ■ ■ ■ . .
. . ■ ■ ■ ■ . . . . . . . . . . . . . . . . . . . . . . . . . . . . ■ ■ ■ ■ . .
. . ■ ■ ■ ■ . . . . . . . . . . . . . . . . . . . . . . . . . . . . ■ ■ ■ ■ . .
. . . . . . . . . . . . . ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ . . . . . . . . . . . . .
. . . . . . . . . . . . . ■ . . . . . . . . . . . . ■ . . . G . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . ■ . . . . . . . . . . . . .
. . . . . . . . . . @ . . . . . . . . . . . . . . . ■ . . . . . . . . . . . . .
. . ■ ■ ■ ■ . . . . . . . . . . . . . . . . . . . . ■ . . . . . . . ■ ■ ■ ■ . .
. . ■ ■ ■ ■ . . . . . . . ■ . . . . . . . . . . . . ■ . . . . . . . ■ ■ ■ ■ . .
. . ■ ■ ■ ■ . . . . . . . ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ ■ . . . . . . . ■ ■ ■ ■ . .
. . . . . . . . . . . . . . . . . . . . 

Each node is stored in the `grid` object, which contains a list of lists containing `Node` objects in the `grid.nodes` attribute.

Example usage of the classes:

In [7]:
print("Initial state:")
start = grid.get_initial()  # initial state at (10, 12), a Node object
print(start)
successors = start.expand() # expand the initial state to generate successors
print("\nAfter expansion:")
print(start)
print("\nSuccessors of the initial state, expanded:")
for successor in successors:
    successor.expand()
    print(successor)
print(f"\nGoal check for first successor of first successor ({start.successors[0].successors[0].coords}):")
print(start.successors[0].successors[0].goal_test())

Initial state:
Node(coords=(10, 12), parent=None, successors=[], initial=True, goal=False)

After expansion:
Node(coords=(10, 12), parent=None, successors=[(10, 13), (9, 12), (10, 11), (11, 12)], initial=True, goal=False)

Successors of the initial state, expanded:
Node(coords=(10, 13), parent=(10, 12), successors=[(10, 14), (9, 13), (11, 13)], initial=False, goal=False)
Node(coords=(9, 12), parent=(10, 12), successors=[(9, 13), (8, 12), (9, 11)], initial=False, goal=False)
Node(coords=(10, 11), parent=(10, 12), successors=[(9, 11), (10, 10), (11, 11)], initial=False, goal=False)
Node(coords=(11, 12), parent=(10, 12), successors=[(11, 13), (11, 11), (12, 12)], initial=False, goal=False)

Goal check for first successor of first successor ((10, 14)):
False


Check the visualization above and take a moment to confirm that the outputs of the print statements match the problem at hand. The coordinates begin at (0, 0) in the bottom left corner of the grid.

**Task 1: Depth-first search (0.4 pt)**

In [ ]:
# ---------- YOUR CODE HERE (OPTIONAL) ----------- #
grid.set_search_visualization(True) # set to False to disable search visualization
grid.set_search_delay(0.1) # adjust delay for search visualization (in seconds)
# ---------- YOUR CODE HERE (OPTIONAL) ----------- #

Your first task is to implement the **depth-first search** (DFS) algorithm. DFS is a simple uninformed search algorithm that explores the search space by always expanding the deepest node in the search tree first.

All the code for visualizing your results has been implemented for you and requires nothing more than calling the `dfs` function. If search visualization is enabled, you can see your agent moving through the search space in real time.*

Fill in the `dfs` function. It should traverse the search space starting from the initial node and return the goal node if it is found, or `None` if it is not found. All the methods you need to use have been introduced in the previous sections.

<small>*You may experience some performance issues with the visualization if you are running this notebook locally on a lower-end machine. If this is the case, we recommend running the code on the CSC platform (or an equivalent) or modifying the parameters in the previous code block.</small>

In [ ]:
def dfs(grid):
    # ---------- YOUR CODE HERE ----------- #
    # 1. Define variables
    # Hint: you will need to keep track of 1) the nodes that have been reached and 2) the frontier. 
    # You can use a set (= unordered list with no duplicate values) for the `reached` variable: reached = set()
    # Is there a suitable data structure for the frontier..?
    raise NotImplementedError("Replace with your code")

    # 2. Push the initial state to the frontier
    
    # 3. Keep taking nodes from the frontier until it is empty

        # 4. If the node is the goal, return it

        # 5. If the node is not the goal and has not been reached, then... what?

    # 6. If the frontier is empty and no goal has been found, return none

    # ---------- YOUR CODE HERE ----------- #

# Perform depth-first search
grid.reset() # cleanup to get rid of intermediate results
result = dfs(grid)

In [ ]:
def visualize_results(grid, result_node):
    """ 
    Visualize the path from the start node to the goal node.
    """
    path_nodes = []
    current = result_node
    
    while current is not None:
        path_nodes.append(current)
        current = current.parent
    path_nodes.reverse()
    
    for node in path_nodes:
        if node is not path_nodes[0] and node is not path_nodes[-1]:
            node.current = True
    grid.visualize()
    
    print(f"Goal found at {result_node.coords} after {grid.expansions} expansions.")
    print("Path from start to goal: ", end="")

    for node in path_nodes:
        if node.action:
            print(f" -> {node.action} -> ", end="")
        print(f"{node.coords}", end="")

# Visualize the path if a result was found
if result:
    visualize_results(grid, result)
else:
    print("No path found to the goal.")

Note that your `dfs` function only needed to return the goal node. In a real-world application you would also need to return the path to the goal, as well as the actions taken to reach it. You may wish to take a look at the `visualize_results` function above as well as the `expand` method of the `Node` class in `ex1_utils.py` to see how the path is reconstructed from the goal node. The basic idea is to keep track of the parent node as well as action taken at each node as you traverse the search space, and then reconstruct the path by following these pointers back to the initial node.

**Task 2: Breath-first search (0.2 pt)**

Your next task is to implement the **breadth-first search** (BFS) algorithm. BFS is another uninformed search algorithm that explores the search space by always expanding the shallowest node in the search tree first.

(Hint: if done correctly, you only need to change one line in the `dfs` function!)

In [ ]:
def bfs(grid):
    # ---------- YOUR CODE HERE ----------- #

    raise NotImplementedError("Replace with your code")

    # ---------- YOUR CODE HERE ----------- #

# Perform breadth-first search
grid.reset() # cleanup to get rid of intermediate results
result = bfs(grid)

In [ ]:
# Visualize the results if a goal was found
if result:
    visualize_results(grid, result)
else:
    print("No path found to the goal.")

### 1.2 – Informed (Heuristic) Search

The uninformed search algorithms we utilized above have no knowledge about the problem domain and therefore cannot make informed decisions about which nodes to expand next. This can lead to inefficient search behavior, especially in large search spaces. Informed search algorithms, on the other hand, use additional information (heuristics) to guide the search process and make it more efficient.

In grid problems such as ours, a common heuristic is the **Manhattan distance**. The Manhattan distance between two points (x1, y1) and (x2, y2) is defined as the sum of the absolute differences of their coordinates. It can be interpreted as the minimum number of moves required to reach the goal state from the current state ignoring obstacles.

In [ ]:
def manhattan(node1, node2):
    """
    Returns the Manhattan distance between two nodes.
    """
    return abs(node1.coords[0] - node2.coords[0]) + abs(node1.coords[1] - node2.coords[1])

print(f"Manhattan distance from start to goal: {manhattan(grid.get_initial(), grid.get_goal())}")

You will also need a **priority queue** for this task. One has been provided for you in `ex1_utils.py`.

In [ ]:
from ex1_utils import PriorityQueue

Usage of the `PriorityQueue` class:

In [ ]:
my_pqueue = PriorityQueue()  # create an instance of the PriorityQueue class
my_pqueue.push("a", 1)  # pqueue is now [("a", 1)]
my_pqueue.push("b", 10)  # pqueue is now [("a", 1), ("b", 10)]
my_pqueue.push("c", 5)  # pqueue is now [("a", 1), ("c", 5), ("b", 10)] (automatically sorted by priority)
print(my_pqueue.read())  # should print "a" (highest priority)
my_pqueue.pop()  # pqueue is now [("c", 5), ("b", 10)]
print(my_pqueue.read())  # should print "c" (next highest priority)
print(my_pqueue.pop())  # also prints "c", pqueue is now [("b", 10)]
my_pqueue.pop() # pqueue is now empty
print(my_pqueue.read())  # should print None, since pqueue is empty

**Task 3: A\* search (0.4 pt)**

Your final task is to implement the **A\*** search algorithm. In this task each node is represented as a tuple containing the node itself and its priority, which is the sum of the path cost and the heuristic value. The path cost `g` is the number of actions taken to reach the node from the start, and the heuristic value `h` is the Manhattan distance from the node to the goal state. The `Node` class has attributes for storing these values:

In [ ]:
my_node = Node(grid)
my_node.g = 10
my_node.h = 5
my_priority = my_node.g + my_node.h
print(my_priority) # should print 10 + 5 = 15

The `a_star` function is similar to the previous ones, but you will need to take into account the priorities of the nodes.

In [ ]:
def a_star(grid):
    # ---------- YOUR CODE HERE ----------- #

    raise NotImplementedError("Replace with your code")

    # ---------- YOUR CODE HERE ----------- #

# Perform A* search
grid.reset()
result = a_star(grid)

In [ ]:
if result:
    visualize_results(grid, result)
else:
    print("No path found to the goal.")

### 1.3 – Summary

In this assignment, you implemented three search algorithms: depth-first search (DFS), breadth-first search (BFS) and A\* search. Fill in the table below with the results of your experiments.

| Algorithm | Nodes expanded | Optimal path found (y/n) |
|-----------|----------------|--------------------------|
| DFS       |                |                          |
| BFS       |                |                          |
| A*        |                |                          |


### EXTRA: Discussion

1. **Which algorithm do you think is best suited for this problem? Why? Can you imagine a scenario where you would prefer the other two algorithms over the one you chose?**
2. **How would switching to a heuristic based on Euclidean distance affect the performance of the A\* algorithm? Why?**

YOUR ANSWER HERE

## Aftermath

Please provide short answers to the following questions:

**1. Did you experience any issues or find anything particularly confusing?**

YOUR ANSWER HERE

**2. Is there anything you would like to see improved in the assignment?**

YOUR ANSWER HERE


### Submission

1. Make sure you have completed all tasks and filled in your personal details at the top of this notebook.
2. Ensure all the code runs without errors: restart the kernel and run all cells in order.
3. Submit *only* this notebook (`ex1.ipynb`) on Moodle.